In [1]:
from sqlalchemy import create_engine, text
import psycopg2
import config
import pandas as pd

In [2]:
db = config.db_config
db

{'username': 'enessariyer',
 'password': '',
 'host': 'localhost',
 'port': '5432',
 'database': 'usa_spending'}

In [3]:
url = f'postgresql+psycopg2://{db['username']}:{db['password']}@{db['host']}:{db['port']}/{db['database']}'
engine = create_engine(url)

In [4]:
# Bağlantı başarılı mı?
with engine.connect() as conn:
    result = conn.execute(text('select 1'))
    print(result.fetchone())

(1,)


In [5]:
# Queries
q1_alici_kumulatif_fon = """WITH alici_ozet AS (
SELECT
		r.recipient_name AS alici_ismi,
		sum(f.federal_action_obligation) AS toplam_fon
FROM
	fact_spending f
JOIN dim_recipient r
	ON
	f.recipient_key = r.recipient_key
GROUP BY
	r.recipient_name
)
SELECT
	alici_ismi,
	toplam_fon,
	sum(toplam_fon) OVER (ORDER BY toplam_fon DESC) / sum(toplam_fon) OVER () * 100 AS kumulatif_yuzde,
	toplam_fon / sum(toplam_fon) OVER () * 100 AS toplamdaki_pay
FROM alici_ozet
ORDER BY toplam_fon DESC 
"""
q2_para_yakma_hizi_MA = """
WITH agg_tablo AS (
SELECT
	date_trunc('month', action_date) AS aylar,
	sum(federal_action_obligation) AS aylik_fon
FROM
	fact_spending
GROUP BY 1
)
SELECT
	to_char(aylar, 'YY-MM') AS donem,
	round(aylik_fon, 2) AS aylik_harcama,
	round(avg(aylik_fon) OVER (ORDER BY aylar ROWS BETWEEN 2 PRECEDING AND CURRENT ROW), 2) AS three_month_ma
FROM agg_tablo
ORDER BY aylar
"""
q3_relative_growth_for_months = """
WITH donemsel_karsilastirma AS (
SELECT
		r.recipient_name AS alici_ismi,
		sum(CASE WHEN date_trunc('month', action_date) = '2023-10-01'
		THEN f.federal_action_obligation ELSE 0 END) AS eski_donem,
		sum(CASE WHEN date_trunc('month', action_date) = '2024-01-01'
		THEN f.federal_action_obligation ELSE 0 END) yeni_donem
FROM
	fact_spending f
JOIN dim_recipient r ON
	f.recipient_key = r.recipient_key
GROUP BY r.recipient_name
)
SELECT
	alici_ismi,
	eski_donem,
	yeni_donem,
	round( (yeni_donem - eski_donem) / NULLIF(eski_donem, 0), 2) * 100 AS buyume_yuzdesi
FROM donemsel_karsilastirma
WHERE eski_donem > 0 AND yeni_donem > 0
ORDER BY buyume_yuzdesi DESC
"""
q4_eylul_patlamasi = """
WITH aylik_ozet AS (
SELECT
	date_trunc('month', action_date) AS aylar,
	sum(federal_action_obligation) AS toplam_harcama
FROM fact_spending
GROUP BY aylar
)
SELECT
	to_char(aylar, 'YY-MM') AS aylar,
	(toplam_harcama - LAG(toplam_harcama) OVER w) / 
	NULLIF(LAG(toplam_harcama) OVER w, 0) * 100 AS buyume
FROM aylik_ozet
WINDOW w AS (ORDER BY aylar)
ORDER BY aylar
"""

In [6]:
analiz1 = pd.read_sql(q1_alici_kumulatif_fon, engine)
analiz2 = pd.read_sql(q2_para_yakma_hizi_MA, engine)
analiz3 = pd.read_sql(q3_relative_growth_for_months, engine)
analiz4 = pd.read_sql(q4_eylul_patlamasi, engine)

In [7]:
print(analiz1.shape)
analiz1.info()
analiz1.head()

(3031, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3031 entries, 0 to 3030
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   alici_ismi       3031 non-null   object 
 1   toplam_fon       3031 non-null   float64
 2   kumulatif_yuzde  3031 non-null   float64
 3   toplamdaki_pay   3031 non-null   float64
dtypes: float64(3), object(1)
memory usage: 94.8+ KB


,alici_ismi,toplam_fon,kumulatif_yuzde,toplamdaki_pay
0,CALIFORNIA INSTITUTE OF TECHNOLOGY,2.227553e+09,11.107810,11.107810
1,SPACE EXPLORATION TECHNOLOGIES CORP.,1.996992e+09,21.065917,9.958107
2,THE BOEING COMPANY,1.515321e+09,28.622144,7.556227
3,LOCKHEED MARTIN CORP,1.133893e+09,34.276361,5.654217
4,"AMENTUM TECHNOLOGY, INC.",1.036168e+09,39.443270,5.166908


In [8]:
analiz1 = analiz1.rename(columns={'toplam_fon':'toplam_fon(milyar_dolar)'})

In [9]:
analiz1['toplam_fon(milyar_dolar)'] = analiz1['toplam_fon(milyar_dolar)'] / 1000000000
analiz1.head()

,alici_ismi,toplam_fon(milyar_dolar),kumulatif_yuzde,toplamdaki_pay
0,CALIFORNIA INSTITUTE OF TECHNOLOGY,2.227553,11.107810,11.107810
1,SPACE EXPLORATION TECHNOLOGIES CORP.,1.996992,21.065917,9.958107
2,THE BOEING COMPANY,1.515321,28.622144,7.556227
3,LOCKHEED MARTIN CORP,1.133893,34.276361,5.654217
4,"AMENTUM TECHNOLOGY, INC.",1.036168,39.443270,5.166908


In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

top_15 = analiz1.head(15)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=top_15['alici_ismi'], y=top_15['toplam_fon(milyar_dolar)'], 
    name='Toplam Fon (Milyar Dolar)'),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=top_15['alici_ismi'], y=top_15['kumulatif_yuzde'], name='Kumulatif %', 
    line=dict(color='red', width=3)),
    secondary_y= True
)

fig.update_layout(title={
    'text':'NASA Fonlarının Firmalara Gore Dagilimi',
    'y':0.95,
    'x':0.43,
    'xanchor':'center',
    'yanchor':'top',
    'font':{
        'color':'darkblue',
        'size':17
    }}, 
    legend=dict(
        x=1.1,
        y=1.,
        font={'color':'darkblue'}
    ), 
    xaxis_tickangle=-60)
fig.update_xaxes(
    tickfont=dict(color='green')
)
fig.update_yaxes(title_text='Fon Miktarı (Milyar Dolar)', 
                title_font=dict(color='red'),
                secondary_y= False,
                title_standoff=10)
fig.update_yaxes(title_text='Kumulatif Pay (%)', 
                title_font=dict(color='red'),
                secondary_y=True, 
                range=[0, 100],
                title_standoff=8)
fig.show()

In [11]:
print(analiz2.shape)
analiz2.info()
analiz2.head()

(12, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   donem           12 non-null     object 
 1   aylik_harcama   12 non-null     float64
 2   three_month_ma  12 non-null     float64
dtypes: float64(2), object(1)
memory usage: 420.0+ bytes


,donem,aylik_harcama,three_month_ma
0,23-10,1.028360e+09,1.028360e+09
1,23-11,1.094811e+09,1.061586e+09
2,23-12,1.623364e+09,1.248845e+09
3,24-01,9.684490e+08,1.228875e+09
4,24-02,1.634331e+09,1.408715e+09


In [12]:
analiz2['aylik_harcama'] = analiz2['aylik_harcama'] / 1000000000
analiz2 = analiz2.rename(columns={'aylik_harcama':'aylik_harcama (MilyarDolar)'})
analiz2.head()

,donem,aylik_harcama (MilyarDolar),three_month_ma
0,23-10,1.028360,1.028360e+09
1,23-11,1.094811,1.061586e+09
2,23-12,1.623364,1.248845e+09
3,24-01,0.968449,1.228875e+09
4,24-02,1.634331,1.408715e+09


In [13]:
analiz2['three_month_ma'] = analiz2['three_month_ma'] / 1000000000
analiz2 = analiz2.rename(columns={'three_month_ma':'three_month_ma_milyardolar'})
analiz2.head()

,donem,aylik_harcama (MilyarDolar),three_month_ma_milyardolar
0,23-10,1.028360,1.028360
1,23-11,1.094811,1.061586
2,23-12,1.623364,1.248845
3,24-01,0.968449,1.228875
4,24-02,1.634331,1.408715


In [14]:
analiz2['donem'] = pd.to_datetime(analiz2['donem'], format='%y-%m')

In [15]:
analiz2.head()

,donem,aylik_harcama (MilyarDolar),three_month_ma_milyardolar
0,2023-10-01,1.028360,1.028360
1,2023-11-01,1.094811,1.061586
2,2023-12-01,1.623364,1.248845
3,2024-01-01,0.968449,1.228875
4,2024-02-01,1.634331,1.408715


In [16]:
from turtle import title


fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=analiz2['donem'],
    y=analiz2['aylik_harcama (MilyarDolar)'],
    name='aylik_harcama'
))

fig2.add_trace(go.Scatter(
    x=analiz2['donem'],
    y=analiz2['three_month_ma_milyardolar'],
    name='aylik_harcama_3Month_ma',
    line=dict(color='red', width=4)
))

fig2.update_layout(
    title=dict(
        text='Harcama Trendi',
        x=0.38,
        y=0.9,
        font=dict(color='orange', size=25))
)

fig2.update_xaxes(
    title_text = 'Ay',
    title_font=dict(color='orange', size=15),
    dtick='M1'
)

fig2.update_yaxes(
    title_text='Aylik Odeme (Milyar Dolar)',
    title_font=dict(color='orange', size=15)
)

fig2.show()


In [17]:
print(analiz3.shape)
analiz3.info()
analiz3.head()

(174, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   alici_ismi      174 non-null    object 
 1   eski_donem      174 non-null    float64
 2   yeni_donem      174 non-null    float64
 3   buyume_yuzdesi  174 non-null    float64
dtypes: float64(3), object(1)
memory usage: 5.6+ KB


,alici_ismi,eski_donem,yeni_donem,buyume_yuzdesi
0,ARES TECHNICAL SERVICES CORPORATION,3552.82,1323000.00,37138.0
1,ASRC FEDERAL FACILITIES LOGISTICS LLC,24967.00,2250859.00,8915.0
2,THE REGENTS OF THE UNIVERSITY OF COLORADO,49685.50,3895786.00,7741.0
3,NATIVE RESOURCE DEVELOPMENT CO INC,13837.70,862896.10,6136.0
4,COLSA CORP,32000.00,1894537.27,5820.0


In [18]:
import plotly.express as px

In [19]:
analiz3_top_15 = analiz3.head(15)

fig3 = px.bar(
    analiz3_top_15, 
    x='alici_ismi', 
    y='buyume_yuzdesi',
    title='iki ay arasindaki harcamadaki buyume',
    labels={'alici_ismi':'sirket', 'buyume_yuzdesi':'buyume'},
    text_auto='.1f',
    )

fig3.update_xaxes(
    title_font = dict(color='red', size=22),
    tickfont = dict(color='green')
)

fig3.update_yaxes(
    title_font = dict(color='red',size=22),
    tickfont = dict(color='green')
)

fig3.show()

In [20]:
print(analiz4.shape)
analiz4.info()
analiz4.head()

(12, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   aylar   12 non-null     object 
 1   buyume  11 non-null     float64
dtypes: float64(1), object(1)
memory usage: 324.0+ bytes


,aylar,buyume
0,23-10,NaN
1,23-11,6.461856
2,23-12,48.277991
3,24-01,-40.343081
4,24-02,68.757543


In [21]:
analiz4['aylar'] = pd.to_datetime(analiz4['aylar'], format='%y-%m')
analiz4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   aylar   12 non-null     datetime64[ns]
 1   buyume  11 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 324.0 bytes


In [22]:
analiz4['durum'] = analiz4['buyume'].apply(lambda x: 'Pozitif' if x>0 else 'Negatif')
analiz4.head()

,aylar,buyume,durum
0,2023-10-01,NaN,Negatif
1,2023-11-01,6.461856,Pozitif
2,2023-12-01,48.277991,Pozitif
3,2024-01-01,-40.343081,Negatif
4,2024-02-01,68.757543,Pozitif


In [23]:
fig4 = px.bar(analiz4, x='aylar', 
y='buyume',
title='Aylık Büyüme',
color='durum',
labels={'buyume':'Buyume Oranı (%)', 'aylar':'Dönem'},
color_discrete_map={'Negatif':'red', 'Pozitif':'green'}
)

fig4.update_layout(
    title={'font':dict(color='darkblue', size=23),
    'x':0.48, 'y':0.92},
    legend=dict(
        title='Durum',
        font=dict(color='darkblue')),
    template='plotly_dark'
)

fig4.update_xaxes(
    dtick='M1'
)

fig4.show()